# Advent of Code

## 2019-012-015
## 2019 015

https://adventofcode.com/2019/day/15

In [1]:
from collections import defaultdict, deque

def run_intcode(program, input_values):
    memory = defaultdict(int, enumerate(program))
    pointer = 0
    relative_base = 0
    outputs = []

    def get_value(parameter, mode):
        if mode == 0:  # Position mode
            return memory[parameter]
        elif mode == 1:  # Immediate mode
            return parameter
        elif mode == 2:  # Relative mode
            return memory[parameter + relative_base]
        else:
            raise ValueError(f"Unknown mode: {mode}")

    def write_value(parameter, mode, value):
        if mode == 0:  # Position mode
            memory[parameter] = value
        elif mode == 2:  # Relative mode
            memory[parameter + relative_base] = value
        else:
            raise ValueError(f"Invalid mode for writing: {mode}")

    input_iterator = iter(input_values)
    while True:
        instruction = str(memory[pointer]).zfill(5)
        opcode = int(instruction[-2:])
        mode1 = int(instruction[-3])
        mode2 = int(instruction[-4])
        mode3 = int(instruction[-5])

        if opcode == 99:  # Halt
            break
        elif opcode in [1, 2]:  # Add or Multiply
            param1 = memory[pointer + 1]
            param2 = memory[pointer + 2]
            param3 = memory[pointer + 3]
            value1 = get_value(param1, mode1)
            value2 = get_value(param2, mode2)
            if opcode == 1:  # Add
                write_value(param3, mode3, value1 + value2)
            elif opcode == 2:  # Multiply
                write_value(param3, mode3, value1 * value2)
            pointer += 4
        elif opcode == 3:  # Input
            param1 = memory[pointer + 1]
            try:
                write_value(param1, mode1, next(input_iterator))
            except StopIteration:
                raise RuntimeError("Input required but not provided")
            pointer += 2
        elif opcode == 4:  # Output
            param1 = memory[pointer + 1]
            outputs.append(get_value(param1, mode1))
            pointer += 2
            yield outputs.pop(0)  # Output one value at a time
        elif opcode in [5, 6]:  # Jump-if-true or Jump-if-false
            param1 = memory[pointer + 1]
            param2 = memory[pointer + 2]
            value1 = get_value(param1, mode1)
            value2 = get_value(param2, mode2)
            if (opcode == 5 and value1 != 0) or (opcode == 6 and value1 == 0):
                pointer = value2
            else:
                pointer += 3
        elif opcode in [7, 8]:  # Less than or Equals
            param1 = memory[pointer + 1]
            param2 = memory[pointer + 2]
            param3 = memory[pointer + 3]
            value1 = get_value(param1, mode1)
            value2 = get_value(param2, mode2)
            if (opcode == 7 and value1 < value2) or (opcode == 8 and value1 == value2):
                write_value(param3, mode3, 1)
            else:
                write_value(param3, mode3, 0)
            pointer += 4
        elif opcode == 9:  # Adjust relative base
            param1 = memory[pointer + 1]
            relative_base += get_value(param1, mode1)
            pointer += 2
        else:
            raise ValueError(f"Unknown opcode {opcode} at position {pointer}")

def explore_area(program):
    directions = {1: (0, -1), 2: (0, 1), 3: (-1, 0), 4: (1, 0)}
    reverse_direction = {1: 2, 2: 1, 3: 4, 4: 3}

    def move(position, direction):
        dx, dy = directions[direction]
        return position[0] + dx, position[1] + dy

    droid = run_intcode(program, [])
    visited = {}
    stack = [(0, 0, None)]  # (current_position, previous_position, move_to_previous)
    visited[(0, 0)] = 1
    oxygen_position = None

    while stack:
        current, previous, backtrack_direction = stack.pop()

        if backtrack_direction:  # Backtrack if needed
            next(droid)  # Resume execution
            next(droid)  # Resume execution again for the response
            continue

        for direction in range(1, 5):
            new_position = move(current, direction)
            if new_position in visited:
                continue

            # Send movement command
            response = next(droid)

            if response == 0:  # Wall
                visited[new_position] = 0
            elif response in {1, 2}:  # Moved successfully
                visited[new_position] = 1
                if response == 2:  # Found oxygen system
                    oxygen_position = new_position
                stack.append((new_position, current, reverse_direction[direction]))
                break

    return visited, oxygen_position


def bfs_shortest_path(visited, start, goal):
    directions = [(0, -1), (0, 1), (-1, 0), (1, 0)]
    queue = deque([(start, 0)])
    visited_set = set()

    while queue:
        position, steps = queue.popleft()
        if position == goal:
            return steps

        if position in visited_set:
            continue
        visited_set.add(position)

        for dx, dy in directions:
            neighbor = (position[0] + dx, position[1] + dy)
            if neighbor in visited and visited[neighbor] == 1:
                queue.append((neighbor, steps + 1))

    return -1


# Load the Intcode program
with open("input.txt") as f:
    program = list(map(int, f.read().strip().split(',')))

# Explore the area and locate the oxygen system
visited, oxygen_position = explore_area(program)

# Calculate the shortest path to the oxygen system
shortest_path = bfs_shortest_path(visited, (0, 0), oxygen_position)
print(f"Shortest path to oxygen system: {shortest_path}")


TypeError: 'int' object is not subscriptable

In [2]:
def explore_area(program):
    directions = {1: (0, -1), 2: (0, 1), 3: (-1, 0), 4: (1, 0)}
    reverse_direction = {1: 2, 2: 1, 3: 4, 4: 3}

    def move(position, direction):
        dx, dy = directions[direction]
        return position[0] + dx, position[1] + dy

    # Intcode generator
    droid = run_intcode(program, [])
    visited = {}
    stack = [(0, 0)]  # Initialize stack with the starting position
    visited[(0, 0)] = 1
    oxygen_position = None

    while stack:
        current = stack.pop()

        for direction in range(1, 5):
            new_position = move(current, direction)
            if new_position in visited:
                continue

            # Send movement command
            output = next(droid)
            if output == 0:  # Wall
                visited[new_position] = 0
            elif output in {1, 2}:  # Moved successfully
                visited[new_position] = 1
                if output == 2:  # Found oxygen system
                    oxygen_position = new_position
                stack.append(new_position)
                # Backtrack to the previous position
                next(droid)  # Send the reverse command to return to the original position

    return visited, oxygen_position


In [3]:
# Load the Intcode program
with open("input.txt") as f:
    program = list(map(int, f.read().strip().split(',')))

# Explore the area and locate the oxygen system
visited, oxygen_position = explore_area(program)

# Calculate the shortest path to the oxygen system
shortest_path = bfs_shortest_path(visited, (0, 0), oxygen_position)
print(f"Shortest path to oxygen system: {shortest_path}")

RuntimeError: Input required but not provided

In [5]:
from collections import defaultdict, deque

def run_intcode(program, input_values):
    memory = defaultdict(int, enumerate(program))
    pointer = 0
    relative_base = 0
    outputs = []

    def get_value(parameter, mode):
        if mode == 0:  # Position mode
            return memory[parameter]
        elif mode == 1:  # Immediate mode
            return parameter
        elif mode == 2:  # Relative mode
            return memory[parameter + relative_base]
        else:
            raise ValueError(f"Unknown mode: {mode}")

    def write_value(parameter, mode, value):
        if mode == 0:  # Position mode
            memory[parameter] = value
        elif mode == 2:  # Relative mode
            memory[parameter + relative_base] = value
        else:
            raise ValueError(f"Invalid mode for writing: {mode}")

    input_iterator = iter(input_values)
    while True:
        instruction = str(memory[pointer]).zfill(5)
        opcode = int(instruction[-2:])
        mode1 = int(instruction[-3])
        mode2 = int(instruction[-4])
        mode3 = int(instruction[-5])

        if opcode == 99:  # Halt
            break
        elif opcode in [1, 2]:  # Add or Multiply
            param1 = memory[pointer + 1]
            param2 = memory[pointer + 2]
            param3 = memory[pointer + 3]
            value1 = get_value(param1, mode1)
            value2 = get_value(param2, mode2)
            if opcode == 1:  # Add
                write_value(param3, mode3, value1 + value2)
            elif opcode == 2:  # Multiply
                write_value(param3, mode3, value1 * value2)
            pointer += 4
        elif opcode == 3:  # Input
            param1 = memory[pointer + 1]
            try:
                write_value(param1, mode1, next(input_iterator))
            except StopIteration:
                raise RuntimeError("Input required but not provided")
            pointer += 2
        elif opcode == 4:  # Output
            param1 = memory[pointer + 1]
            outputs.append(get_value(param1, mode1))
            pointer += 2
            yield outputs.pop(0)  # Output one value at a time
        elif opcode in [5, 6]:  # Jump-if-true or Jump-if-false
            param1 = memory[pointer + 1]
            param2 = memory[pointer + 2]
            value1 = get_value(param1, mode1)
            value2 = get_value(param2, mode2)
            if (opcode == 5 and value1 != 0) or (opcode == 6 and value1 == 0):
                pointer = value2
            else:
                pointer += 3
        elif opcode in [7, 8]:  # Less than or Equals
            param1 = memory[pointer + 1]
            param2 = memory[pointer + 2]
            param3 = memory[pointer + 3]
            value1 = get_value(param1, mode1)
            value2 = get_value(param2, mode2)
            if (opcode == 7 and value1 < value2) or (opcode == 8 and value1 == value2):
                write_value(param3, mode3, 1)
            else:
                write_value(param3, mode3, 0)
            pointer += 4
        elif opcode == 9:  # Adjust relative base
            param1 = memory[pointer + 1]
            relative_base += get_value(param1, mode1)
            pointer += 2
        else:
            raise ValueError(f"Unknown opcode {opcode} at position {pointer}")

# Load the Intcode program
with open("input.txt") as f:
    program = list(map(int, f.read().strip().split(',')))

# Explore the area and locate the oxygen system
visited, oxygen_position = explore_area(program)

# Calculate the shortest path to the oxygen system
def bfs_shortest_path(visited, start, goal):
    directions = [(0, -1), (0, 1), (-1, 0), (1, 0)]
    queue = deque([(start, 0)])
    visited_set = set()

    while queue:
        position, steps = queue.popleft()
        if position == goal:
            return steps

        if position in visited_set:
            continue
        visited_set.add(position)

        for dx, dy in directions:
            neighbor = (position[0] + dx, position[1] + dy)
            if neighbor in visited and visited[neighbor] == 1:
                queue.append((neighbor, steps + 1))

    return -1

shortest_path = bfs_shortest_path(visited, (0, 0), oxygen_position)
print(f"Shortest path to oxygen system: {shortest_path}")

RuntimeError: Input required but not provided

In [1]:
from collections import defaultdict, deque

def run_intcode(program, input_values):
    memory = defaultdict(int, enumerate(program))
    pointer = 0
    relative_base = 0
    outputs = []

    def get_value(parameter, mode):
        if mode == 0:  # Position mode
            return memory[parameter]
        elif mode == 1:  # Immediate mode
            return parameter
        elif mode == 2:  # Relative mode
            return memory[parameter + relative_base]
        else:
            raise ValueError(f"Unknown mode: {mode}")

    def write_value(parameter, mode, value):
        if mode == 0:  # Position mode
            memory[parameter] = value
        elif mode == 2:  # Relative mode
            memory[parameter + relative_base] = value
        else:
            raise ValueError(f"Invalid mode for writing: {mode}")

    input_iterator = iter(input_values)
    while True:
        instruction = str(memory[pointer]).zfill(5)
        opcode = int(instruction[-2:])
        mode1 = int(instruction[-3])
        mode2 = int(instruction[-4])
        mode3 = int(instruction[-5])

        if opcode == 99:  # Halt
            break
        elif opcode in [1, 2]:  # Add or Multiply
            param1 = memory[pointer + 1]
            param2 = memory[pointer + 2]
            param3 = memory[pointer + 3]
            value1 = get_value(param1, mode1)
            value2 = get_value(param2, mode2)
            if opcode == 1:  # Add
                write_value(param3, mode3, value1 + value2)
            elif opcode == 2:  # Multiply
                write_value(param3, mode3, value1 * value2)
            pointer += 4
        elif opcode == 3:  # Input
            param1 = memory[pointer + 1]
            try:
                write_value(param1, mode1, next(input_iterator))
            except StopIteration:
                raise RuntimeError("Input required but not provided")
            pointer += 2
        elif opcode == 4:  # Output
            param1 = memory[pointer + 1]
            outputs.append(get_value(param1, mode1))
            pointer += 2
            yield outputs.pop(0)  # Output one value at a time
        elif opcode in [5, 6]:  # Jump-if-true or Jump-if-false
            param1 = memory[pointer + 1]
            param2 = memory[pointer + 2]
            value1 = get_value(param1, mode1)
            value2 = get_value(param2, mode2)
            if (opcode == 5 and value1 != 0) or (opcode == 6 and value1 == 0):
                pointer = value2
            else:
                pointer += 3
        elif opcode in [7, 8]:  # Less than or Equals
            param1 = memory[pointer + 1]
            param2 = memory[pointer + 2]
            param3 = memory[pointer + 3]
            value1 = get_value(param1, mode1)
            value2 = get_value(param2, mode2)
            if (opcode == 7 and value1 < value2) or (opcode == 8 and value1 == value2):
                write_value(param3, mode3, 1)
            else:
                write_value(param3, mode3, 0)
            pointer += 4
        elif opcode == 9:  # Adjust relative base
            param1 = memory[pointer + 1]
            relative_base += get_value(param1, mode1)
            pointer += 2
        else:
            raise ValueError(f"Unknown opcode {opcode} at position {pointer}")

def explore_area(program):
    directions = {1: (0, -1), 2: (0, 1), 3: (-1, 0), 4: (1, 0)}
    reverse_direction = {1: 2, 2: 1, 3: 4, 4: 3}

    def move(position, direction):
        dx, dy = directions[direction]
        return position[0] + dx, position[1] + dy

    input_queue = deque()
    droid = run_intcode(program, input_queue)
    visited = {}
    stack = [(0, 0)]  # Initialize stack with the starting position
    visited[(0, 0)] = 1
    oxygen_position = None

    while stack:
        current = stack.pop()

        for direction in range(1, 5):
            new_position = move(current, direction)
            if new_position in visited:
                continue

            # Send movement command
            input_queue.append(direction)
            output = next(droid)
            if output == 0:  # Wall
                visited[new_position] = 0
            elif output in {1, 2}:  # Moved successfully
                visited[new_position] = 1
                if output == 2:  # Found oxygen system
                    oxygen_position = new_position
                stack.append(new_position)
                # Backtrack to the previous position
                input_queue.append(reverse_direction[direction])
                next(droid)  # Execute the backtrack

    return visited, oxygen_position


def bfs_shortest_path(visited, start, goal):
    directions = [(0, -1), (0, 1), (-1, 0), (1, 0)]
    queue = deque([(start, 0)])
    visited_set = set()

    while queue:
        position, steps = queue.popleft()
        if position == goal:
            return steps

        if position in visited_set:
            continue
        visited_set.add(position)

        for dx, dy in directions:
            neighbor = (position[0] + dx, position[1] + dy)
            if neighbor in visited and visited[neighbor] == 1:
                queue.append((neighbor, steps + 1))

    return -1


# Load the Intcode program
with open("input.txt") as f:
    program = list(map(int, f.read().strip().split(',')))

# Explore the area and locate the oxygen system
visited, oxygen_position = explore_area(program)

# Calculate the shortest path to the oxygen system
shortest_path = bfs_shortest_path(visited, (0, 0), oxygen_position)
print(f"Shortest path to oxygen system: {shortest_path}")

RuntimeError: deque mutated during iteration